In [1]:
from helpers_regs import *
import pandas as pd
import os

In [11]:
#General params
name_mapping = {'vol_vwretd': 'crsp', 'vol_btreturn': 'bonds'}
#lag_list = [12, 9, 6, 3, 1]
lag_list = [12]
dummies = True
date_var  = 'date'
freq_d = 'q' # 'q' for quarterly or 'm' for monthly dummies
alpha_test = 0.10 # 10% significance level
error_mode = 'HAC' # 'HAC' or 'BTP'
n_btsp = None
replacements = {r'cvol_': '', 'gjr_': '', 'gar_': '', r' - (?:[SL]_)?MFH| - SFH': '', ' - TOT': ''}
first_row_for_co = 'United States'
cvol_data = 'cvols_x13_vals_pct_change_1960_2024.dta'
crsp_data = 'CRSP_vol_040523.dta'
bond_data = 'DOW_corp_TRbond_vol_051623.dta'
#Table params
save_table = False
table_filename = 'regressions_midpoint_vals_1960_2024.csv'
#Plot params
save_plot_opt = True
show_opt = False
separation = True
cols_ci = [f'LB ({100-100*alpha_test:.2f}% ci)',f'UB ({100-100*alpha_test:.2f}% ci)']
plot_dir = 'Plots_midpoint_vals_1960_2024/'

In [12]:
data = merge_volatility_data(cvol_data, crsp_data, bond_data)
data = data[data['date'].dt.year < 2020]
data.columns = [i.replace('_p_', ' ').replace('__', ' - ') for i in data.columns]
cols = classify_cols_garch(data)

In [13]:
aux_tables = []
for division, dep_vars in cols.items():
    for var in ['vol_vwretd', 'vol_btreturn']:
        tables = make_regression_tables(data, dep_vars, var, lag_list, dummies, freq_d, date_var, alpha_test, error_mode, n_btsp, replacements, first_row_for_co)
        for t in tables: t.insert(1, 'reg_spec', division + '_' + name_mapping[var])
        aux_tables.append(tables)

In [15]:
full_tables = pd.concat([item for sublist in aux_tables for item in sublist]).reset_index(drop = True)
if save_table: full_tables.to_csv(table_filename, index = False)
full_tables

,dependent variable,reg_spec,n_lags,value,t-statistic,significance,LB (90.00% ci),UB (90.00% ci),p_val,index
0,United States,gar_sfh_crsp,12,-0.000259,-3.262765,1,-0.000389,-0.000128,0.001158,0
1,Connecticut,gar_sfh_crsp,12,0.023155,1.278056,0,-0.006686,0.052996,0.201662,1
2,Georgia,gar_sfh_crsp,12,0.016327,1.323209,0,-0.003996,0.036651,0.186207,2
3,Illinois,gar_sfh_crsp,12,0.007793,2.000869,1,0.001378,0.014208,0.045800,3
4,Michigan,gar_sfh_crsp,12,0.007445,0.792414,0,-0.008030,0.022919,0.428393,4
...,...,...,...,...,...,...,...,...,...,...
843,Texas,gjr_lmfh_bonds,12,-0.000040,-0.336928,0,-0.000235,0.000155,0.736274,19
844,Indiana,gjr_lmfh_bonds,12,-0.000211,-0.567997,0,-0.000823,0.000401,0.570223,20
845,Arizona,gjr_lmfh_bonds,12,-0.000229,-3.343242,1,-0.000343,-0.000116,0.000873,21
846,California,gjr_lmfh_bonds,12,-0.000376,-0.658053,0,-0.001315,0.000564,0.510725,22


In [98]:
os.system(f'mkdir -p {plot_dir}')

In [100]:
for reg_spec in full_tables.reg_spec.unique():
    plim = [1.1*full_tables[(full_tables.reg_spec == reg_spec)].filter(regex=r'LB').values.min(), 
            1.1*full_tables[(full_tables.reg_spec == reg_spec)].filter(regex=r'UB').values.max()]
    for n_lags in lag_list:
        data_plot = full_tables[(full_tables.n_lags == n_lags) & (full_tables.reg_spec == reg_spec)]
        save_filename = f'{plot_dir}{reg_spec}_{error_mode}_{n_lags}mo_{freq_d}_dummies_{int(100*alpha_test)}%ci.pdf'
        generate_bar_plot(data_plot, n_lags, alpha_test, plim, cols_ci[0], cols_ci[1], dummies, freq_d, save_plot_opt, save_filename, show_opt, separation)